In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/1.3_emotion_input_output

/content/drive/MyDrive/1.3_emotion_input_output


In [ ]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

import os
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd

from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split

In [ ]:
dataset_folder='/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48'
sub_folders=os.listdir(dataset_folder)

sub_folders

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']

In [ ]:
i=0
last=[]
images=[]
labels=[]
temp = sub_folders

for sub_folder in sub_folders:
  sub_folder_index = temp.index(sub_folder)
  label = sub_folder_index

  if  label in [4, 6]:
    new_label=0
  elif label in [0,5]:
    new_label=1
  else:
    new_label=2


  path = dataset_folder+'/'+sub_folder
  sub_folder_images= os.listdir(path)

  for image in sub_folder_images:
    image_path = path+'/'+image
    print(image_path+"\t"+str(new_label))

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image= cv2.resize(image,(48,48))
    images.append(image)
    labels.append(new_label)
    i+=1
  last.append(i)

/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S026_003_00000013.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S022_005_00000031.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S011_004_00000020.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S014_003_00000029.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S014_003_00000028.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S014_003_00000030.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S011_004_00000021.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S022_005_00000030.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S022_005_00000032.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S011_004_00000019.png	1
/content/drive/MyDrive/1.3_emotion_input_output/input/CK+48/anger/S010_004_00000019.png	1
/content/d

In [ ]:
images_x = np.array(images)
labels_y = np.array(labels)

images_x = images_x/255

In [ ]:
images_x.shape

(981, 48, 48)

In [ ]:
num_of_classes = 3
labels_y_encoded = tf.keras.utils.to_categorical(labels_y,num_classes=num_of_classes)

In [ ]:
X_train, X_test, Y_train, Y_test= train_test_split(images_x, labels_y_encoded,test_size=0.25, random_state=10)

In [ ]:
len(X_train)

735

In [ ]:
len(X_test)

246

In [ ]:
input = Input(shape = (48,48,1))

conv1 = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(input)
conv1 = Dropout(0.1)(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size = (2,2)) (conv1)

conv2 = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool1)
conv2 = Dropout(0.1)(conv2)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size = (2,2)) (conv2)

conv3 = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool2)
conv3 = Dropout(0.1)(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size = (2,2)) (conv3)

conv4 = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool3)
conv4 = Dropout(0.1)(conv4)
conv4 = Activation('relu')(conv4)
pool4 = MaxPooling2D(pool_size = (2,2)) (conv4)

flatten = Flatten()(pool4)

dense_1 = Dense(128,activation='relu')(flatten)

drop_1 = Dropout(0.2)(dense_1)

output = Dense(3,activation="sigmoid")(drop_1)

In [ ]:
model = Model(inputs=input,outputs=output)
model.compile(optimizer="adam", loss=["categorical_crossentropy"], metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 dropout (Dropout)           (None, 48, 48, 32)        0         
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                             

In [ ]:
fle_s='/content/drive/MyDrive/1.3_emotion_input_output/output/emotion_model.h5'
checkpointer = ModelCheckpoint(fle_s, monitor='loss',verbose=1,save_best_only=True,save_weights_only=False, mode='auto',save_freq='epoch')
callback_list=[checkpointer]

In [ ]:
save = model.fit(X_train,Y_train,batch_size=32,validation_data=(X_test,Y_test),epochs=50,callbacks=[callback_list])

Epoch 1/50
23/23 [==============================] - ETA: 0s - loss: 0.1510 - accuracy: 0.9796
Epoch 1: loss did not improve from 0.12135
23/23 [==============================] - 8s 353ms/step - loss: 0.1510 - accuracy: 0.9796 - val_loss: 0.1683 - val_accuracy: 0.9675
Epoch 2/50
23/23 [==============================] - ETA: 0s - loss: 0.1475 - accuracy: 0.9837
Epoch 2: loss did not improve from 0.12135
23/23 [==============================] - 7s 295ms/step - loss: 0.1475 - accuracy: 0.9837 - val_loss: 0.2050 - val_accuracy: 0.9553
Epoch 3/50
23/23 [==============================] - ETA: 0s - loss: 0.1200 - accuracy: 0.9959
Epoch 3: loss improved from 0.12135 to 0.12005, saving model to /content/drive/MyDrive/1.3_emotion_input_output/output/emotion_model.h5
23/23 [==============================] - 6s 271ms/step - loss: 0.1200 - accuracy: 0.9959 - val_loss: 0.1517 - val_accuracy: 0.9797
Epoch 4/50
23/23 [==============================] - ETA: 0s - loss: 0.1208 - accuracy: 0.9905
Epoch 4: 

In [ ]:
model_path = '/content/drive/MyDrive/1.3_emotion_input_output/output/emotion_model.h5'
loaded_model = load_model(model_path)

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray_img, (48, 48))
    normalized_img = resized_img / 255.0
    input_img = np.expand_dims(normalized_img, axis=-1)

    return input_img

def predict_emotion(image_path):
    input_img = preprocess_image(image_path)
    prediction = loaded_model.predict(np.expand_dims(input_img, axis=0))
    emotion_labels = ['Positive', 'Negative', 'Neutral']
    predicted_label = emotion_labels[np.argmax(prediction)]

    return predicted_label

test_image_path = '/content/im3.jpg'
predicted_emotion = predict_emotion(test_image_path)

print(f'The predicted emotion is: {predicted_emotion}')


1/1 [==============================] - 0s 154ms/step
The predicted emotion is: Negative
